### Build a Simple LLM Application with LCEL
In this quickstart we'll show you how to build a simple LLM application with LangChain. This application will translate text from English into another language. This is a relatively simple LLM application - it's just a single LLM call plus some prompting. Still, this is a great way to get started with LangChain - a lot of features can be built with just some prompting and an LLM call!

After seeing this video, you'll have a high level overview of:

- Using language models

- Using PromptTemplates and OutputParsers

- Using LangChain Expression Language (LCEL) to chain components together

- Debugging and tracing your application using LangSmith

- Deploying your application with LangServe

In [21]:
import os
from dotenv import load_dotenv
load_dotenv()

True

In [22]:
os.environ["GROQ_API_KEY"] = os.getenv("GROQ_API_KEY")
os.environ["LANGCHAIN_API_KEY"] = os.getenv("LANGCHAIN_API_KEY")
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_PROJECT"] = os.getenv("LANGCHAIN_PROJECT")

In [23]:
from langchain_groq import ChatGroq
llm = ChatGroq(model="llama-3.1-8b-instant")
llm

ChatGroq(profile={'max_input_tokens': 131072, 'max_output_tokens': 8192, 'image_inputs': False, 'audio_inputs': False, 'video_inputs': False, 'image_outputs': False, 'audio_outputs': False, 'video_outputs': False, 'reasoning_output': False, 'tool_calling': True}, client=<groq.resources.chat.completions.Completions object at 0x000001EF7EEC2990>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x000001EF7EEC3390>, model_name='llama-3.1-8b-instant', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [24]:
from langchain_core.messages import HumanMessage, SystemMessage

messages = [
    SystemMessage(content="Translate the following text from English to French"),
    HumanMessage(content="Hello, How are you?")
]

response = llm.invoke(messages)
response

AIMessage(content='Bonjour, comment allez-vous ?', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 8, 'prompt_tokens': 49, 'total_tokens': 57, 'completion_time': 0.017410793, 'completion_tokens_details': None, 'prompt_time': 0.00541487, 'prompt_tokens_details': None, 'queue_time': 0.28548212, 'total_time': 0.022825663}, 'model_name': 'llama-3.1-8b-instant', 'system_fingerprint': 'fp_6b5c123dd9', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None, 'model_provider': 'groq'}, id='lc_run--019bd137-7734-7081-b19a-e7544b640c5d-0', tool_calls=[], invalid_tool_calls=[], usage_metadata={'input_tokens': 49, 'output_tokens': 8, 'total_tokens': 57})

In [25]:
from langchain_core.output_parsers import StrOutputParser
output_parser = StrOutputParser()
output_parser.invoke(response)

'Bonjour, comment allez-vous ?'

### LCEL - Chain the Components

In [26]:
chain = llm|output_parser
chain.invoke(messages)

'Bonjour, comment ça va ?'

In [27]:
from langchain_core.prompts import ChatPromptTemplate

generic_template = "You are a language translation expert that translates provided text to {language}"

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", generic_template),
        ("user", "{text}")
    ]
)

In [28]:
result = prompt.invoke({"language" : "Hindi", "text" : "Hello, how are you?"})

In [29]:
result.to_messages()

[SystemMessage(content='You are a language translation expert that translates provided text to Hindi', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='Hello, how are you?', additional_kwargs={}, response_metadata={})]

In [30]:
chain = prompt|llm|output_parser
chain.invoke({"language":"Hindi", "text":"Hello, how are you?"})

'नमस्ते! मैं ठीक हूँ, धन्यवाद।'